In [0]:
# -------------------------------
# Parameters
# -------------------------------
start_m            = "2026M01"  #target
end_m              = "2026M06"  #target
monthwide_start_m  = "2026M01"  #cnos
monthwide_end_m    = "2026M01"  #cnos

# -------------------------------
# Dynamic FY Half -> Quarters
# -------------------------------
def month_num(mstr: str) -> int:
    return int(mstr[-2:])  # "2026M06" -> 6

_selected_month_num = month_num(end_m)

# FY quarters mapping:
# FY 1st half (Jul-Dec)  -> JAS, OND
# FY 2nd half (Jan-Jun)  -> JFM, AMJ
if 1 <= _selected_month_num <= 6:
    fy_half_quarters = ["JFM", "AMJ"]   # 2nd half
else:
    fy_half_quarters = ["JAS", "OND"]   # 1st half

quarter_in_sql = ", ".join([f"'{q}'" for q in fy_half_quarters])

quarter_iya_sql = ",\n    ".join([
    f"MAX(CASE WHEN q.quarter = '{q}' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS {q}_IYA"
    for q in fy_half_quarters
])

final_quarter_cols_sql = ",\n  ".join([
    f"ROUND({q}_IYA, 0) AS {q}_IYA"
    for q in fy_half_quarters
])

# -------------------------------
# Dynamic HALF label (for output names only)
# -------------------------------
half_label = "2H" if (1 <= _selected_month_num <= 6) else "1H"

# Part A: HALF summary columns (must appear before month columns)
final_half_summary_sql = ",\n  ".join([
    f"ROUND(`1H_MM`, 0) AS `{half_label}_MM`",
    f"ROUND(`1H_IYA`, 0) AS `{half_label}_IYA`"
])

# Part B: HALF to-date + comparisons (must appear after month columns)
final_half_tail_sql = ",\n  ".join([
    f"ROUND(`1H_To_Date_MM`, 0) AS `{half_label}_To_Date_MM`",
    f"ROUND(`1H_To_Date_IYA`, 0) AS `{half_label}_To_Date_IYA`",
    f"ROUND(`1H_To_Date_IYA` - `1H_IYA`, 0) AS `vs_Target`",
    f"ROUND(`GIV_1H_to_date_IYA`, 0) AS `GIV_{half_label}_to_date_IYA`",
    f"ROUND(`1H_To_Date_IYA` - `GIV_1H_to_date_IYA`, 0) AS `Gap_VS._C-NOS`"
])

# -------------------------------
# Helpers
# -------------------------------
def month_to_label(mstr: str) -> str:
    mm = mstr[-2:]
    return {
        "01": "JAN", "02": "FEB", "03": "MAR", "04": "APR",
        "05": "MAY", "06": "JUN", "07": "JULY", "08": "AUG",
        "09": "SEP", "10": "OCT", "11": "NOV", "12": "DEC"
    }[mm]

def next_month(mstr: str) -> str:
    yyyy = int(mstr[:4])
    mm   = int(mstr[-2:])
    mm += 1
    if mm == 13:
        yyyy += 1
        mm = 1
    return f"{yyyy}M{mm:02d}"

def months_between(start_m: str, end_m: str):
    out, cur = [], start_m
    while True:
        out.append(cur)
        if cur == end_m:
            break
        cur = next_month(cur)
    return out

# Build month window (inclusive)
months_mw = months_between(monthwide_start_m, monthwide_end_m)

# 1) month_wide for Category (MAX(...))
mw_cols_category_list = []
for m in months_mw:
    lab = month_to_label(m)
    mw_cols_category_list.append(
        f"MAX(CASE WHEN month = '{m}' THEN cnos END)/1000/1000/150.1 AS {lab}_MM"
    )
    mw_cols_category_list.append(
        f"MAX(CASE WHEN month = '{m}' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS {lab}_IYA"
    )
mw_cols_category_sql = ",\n    ".join(mw_cols_category_list)

# 2) 1H_to_date window (Category) (internal names kept as 1H_*)
in_list = ",".join([f"'{m}'" for m in months_mw])
oneH_to_date_mm_sql  = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END)/1000/1000/150.1 AS 1H_to_date"
)
oneH_to_date_iya_sql = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END) / "
    f"NULLIF(SUM(CASE WHEN month IN ({in_list}) THEN cnos_ya END),0)*100 AS 1H_to_date_IYA"
)

# 3) month_wide for TOTAL (SUM(...))
mw_cols_total_list = []
for m in months_mw:
    lab = month_to_label(m)
    mw_cols_total_list.append(
        f"SUM(CASE WHEN month = '{m}' THEN cnos END)/1000/1000/150.1 AS {lab}_MM"
    )
    mw_cols_total_list.append(
        f"SUM(CASE WHEN month = '{m}' THEN cnos END) / "
        f"NULLIF(SUM(CASE WHEN month = '{m}' THEN cnos_ya END),0) * 100 AS {lab}_IYA"
    )
mw_cols_total_sql = ",\n    ".join(mw_cols_total_list)

t_oneH_to_date_mm_sql  = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END)/1000/1000/150.1 AS 1H_to_date"
)
t_oneH_to_date_iya_sql = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END) / "
    f"NULLIF(SUM(CASE WHEN month IN ({in_list}) THEN cnos_ya END),0)*100 AS 1H_to_date_IYA"
)

# 4) Aliases in category_metrics / total_metrics (JULY_MM -> JUL_MM, etc.)
mw_labels = [month_to_label(m) for m in months_mw]
cat_month_mm_select_sql  = ", ".join([f"m.{lab}_MM AS {lab[:3]}_MM" for lab in mw_labels])
cat_month_iya_select_sql = ", ".join([f"m.{lab}_IYA AS {lab[:3]}_IYA" for lab in mw_labels])

# 5) GROUP BY stability lists (exact names from month_wide)
groupby_month_cols_sql = ", ".join([f"m.{lab}_MM, m.{lab}_IYA" for lab in mw_labels])

# 6) GIV to-date window (internal name kept as GIV_1H_*)
giv_1h_to_date_sql = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN giv END) / "
    f"NULLIF(SUM(CASE WHEN month IN ({in_list}) THEN giv_ya END),0) * 100 AS GIV_1H_to_date_IYA"
)

# 7) Final select month columns **as pairs**
final_month_pairs_list = []
for lab in mw_labels:
    final_month_pairs_list.append(f"ROUND({lab[:3]}_MM, 0)  AS {lab[:3]}_MM")
    final_month_pairs_list.append(f"ROUND({lab[:3]}_IYA, 0) AS {lab[:3]}_IYA")
final_month_pairs_sql = ", ".join(final_month_pairs_list)

# -------------------------------
# Final query (dynamic quarters + dynamic HALF NAMES + expected column order)
# -------------------------------
query = f"""
WITH base_cnos AS (
  SELECT
    category,
    quarter,
    month,
    CAST(SUBSTRING(month, 6, 2) AS INT) AS month_num,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(target           AS DOUBLE) END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(ya_actual        AS DOUBLE) END) AS cnos_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month >= '{start_m}' AND month <= '{end_m}'
    AND quarter IN ({quarter_in_sql})
  GROUP BY category, quarter, month
),

quarter_rollup AS (
  SELECT category, quarter,
         SUM(cnos) AS q_cnos,
         SUM(cnos_target) AS q_target,
         SUM(cnos_ya) AS q_ya
  FROM base_cnos
  GROUP BY category, quarter
),

half_year AS (
  SELECT category,
         SUM(cnos) AS hy_cnos,
         SUM(cnos_target) AS hy_target,
         SUM(cnos_ya) AS hy_ya
  FROM base_cnos
  GROUP BY category
),

month_wide AS (
  SELECT
    category,
    {mw_cols_category_sql},
    {oneH_to_date_mm_sql},
    {oneH_to_date_iya_sql}
  FROM base_cnos
  GROUP BY category
),

category_metrics AS (
  SELECT
    m.category,
    {quarter_iya_sql},
    (h.hy_target)/1000/1000/150.1                                          AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                     AS 1H_IYA,
    {cat_month_mm_select_sql},
    {cat_month_iya_select_sql},
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM month_wide m
  LEFT JOIN quarter_rollup q ON q.category = m.category
  LEFT JOIN half_year      h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           {groupby_month_cols_sql},
           m.1H_to_date, m.1H_to_date_IYA
),

t_quarter AS (
  SELECT 'TOTAL' AS category, quarter,
         SUM(cnos) AS q_cnos,
         SUM(cnos_target) AS q_target,
         SUM(cnos_ya) AS q_ya
  FROM base_cnos
  GROUP BY quarter
),

t_half_year AS (
  SELECT 'TOTAL' AS category,
         SUM(cnos) AS hy_cnos,
         SUM(cnos_target) AS hy_target,
         SUM(cnos_ya) AS hy_ya
  FROM base_cnos
),

t_month_wide AS (
  SELECT
    'TOTAL' AS category,
    {mw_cols_total_sql},
    {t_oneH_to_date_mm_sql},
    {t_oneH_to_date_iya_sql}
  FROM base_cnos
),

total_metrics AS (
  SELECT
    m.category,
    {quarter_iya_sql},
    (h.hy_target)/1000/1000/150.1                                          AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                     AS 1H_IYA,
    {cat_month_mm_select_sql},
    {cat_month_iya_select_sql},
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM t_month_wide m
  LEFT JOIN t_quarter q ON q.category = m.category
  LEFT JOIN t_half_year h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           {groupby_month_cols_sql},
           m.1H_to_date, m.1H_to_date_IYA
),

giv_category AS (
  SELECT
    category,
    {giv_1h_to_date_sql}
  FROM (
    SELECT
      category,
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '{start_m}' AND month <= '{end_m}'
    GROUP BY category, month
  ) s
  GROUP BY category
),

giv_total AS (
  SELECT
    'TOTAL' AS category,
    {giv_1h_to_date_sql}
  FROM (
    SELECT
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '{start_m}' AND month <= '{end_m}'
    GROUP BY month
  ) s
),

giv_union AS (
  SELECT * FROM giv_category
  UNION ALL
  SELECT * FROM giv_total
),

final_union AS (
  SELECT
    f.*,
    g.GIV_1H_to_date_IYA
  FROM (
    SELECT * FROM category_metrics
    UNION ALL
    SELECT * FROM total_metrics
  ) f
  LEFT JOIN giv_union g
    ON g.category = f.category
)

SELECT
  category as `By Category`,
  {final_quarter_cols_sql},
  {final_half_summary_sql},
  {final_month_pairs_sql},
  {final_half_tail_sql}
FROM final_union
WHERE category IN (
  'TOTAL','Detergent','Fabric Enhancer','Dish','Air','Baby','Fem','Hair','Shave','Oral','Braun','POC'
)
ORDER BY
  CASE category
    WHEN 'TOTAL'           THEN 0
    WHEN 'Detergent'       THEN 1
    WHEN 'Fabric Enhancer' THEN 2
    WHEN 'Dish'            THEN 3
    WHEN 'Air'             THEN 4
    WHEN 'Baby'            THEN 5
    WHEN 'Fem'             THEN 6
    WHEN 'Hair'            THEN 7
    WHEN 'Shave'           THEN 8
    WHEN 'Oral'            THEN 9
    WHEN 'Braun'           THEN 10
    WHEN 'POC'             THEN 11
    ELSE 99
  END ASC
"""

display(spark.sql(query))

In [0]:
# %python
# -------------------------------
# Parameters (EDIT THESE)
# -------------------------------
start_m            = "2026M01"   # target overall filter start (inclusive)
end_m              = "2026M06"   # target overall filter end   (inclusive)
monthwide_start_m  = "2026M01"   # cnos month-wide breakdown start (drives dynamic columns)
monthwide_end_m    = "2026M01"   # cnos month-wide breakdown end

# -------------------------------
# Dynamic FY Half -> Quarters (NEW)
# -------------------------------
def month_num(mstr: str) -> int:
    return int(mstr[-2:])  # "2026M06" -> 6

_selected_month_num = month_num(end_m)

# FY quarters mapping:
# FY 1st half (Jul-Dec)  -> JAS, OND
# FY 2nd half (Jan-Jun)  -> JFM, AMJ
if 1 <= _selected_month_num <= 6:
    fy_half_quarters = ["JFM", "AMJ"]   # FY 2H
else:
    fy_half_quarters = ["JAS", "OND"]   # FY 1H

quarter_in_sql = ", ".join([f"'{q}'" for q in fy_half_quarters])

# Quarter metric columns (dynamic) — produces JFM_IYA/AMJ_IYA or JAS_IYA/OND_IYA
quarter_iya_sql = ",\n    ".join([
    f"MAX(CASE WHEN q.quarter = '{q}' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS {q}_IYA"
    for q in fy_half_quarters
])

# Final SELECT quarter columns (dynamic, prefixed with c.)
final_quarter_cols_sql = ",\n  ".join([
    f"ROUND(c.{q}_IYA, 0) AS {q}_IYA"
    for q in fy_half_quarters
])

# -------------------------------
# Dynamic HALF label for column names only (NEW)
# -------------------------------
half_label = "2H" if (1 <= _selected_month_num <= 6) else "1H"

# Part A: HALF summary columns (must appear before month columns)
final_half_summary_sql = ",\n  ".join([
    f"ROUND(c.`1H_MM`, 0)  AS `{half_label}_MM`",
    f"ROUND(c.`1H_IYA`, 0) AS `{half_label}_IYA`"
])

# Part B: HALF to-date + comparisons (must appear after month columns)
final_half_tail_sql = ",\n  ".join([
    f"ROUND(c.`1H_To_Date_MM`, 0)  AS `{half_label}_To_Date_MM`",
    f"ROUND(c.`1H_To_Date_IYA`, 0) AS `{half_label}_To_Date_IYA`",
    f"ROUND(c.`1H_To_Date_IYA` - c.`1H_IYA`, 0) AS `vs_Target`",
    f"ROUND(g.GIV_1H_to_date_IYA, 0) AS `GIV_{half_label}_to_date_IYA`",
    f"ROUND(c.`1H_To_Date_IYA` - g.GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`"
])

# -------------------------------
# Helpers
# -------------------------------
def month_to_label(mstr: str) -> str:
    mm = mstr[-2:]
    return {
        "01": "JAN", "02": "FEB", "03": "MAR", "04": "APR",
        "05": "MAY", "06": "JUN", "07": "JULY", "08": "AUG",
        "09": "SEP", "10": "OCT", "11": "NOV", "12": "DEC"
    }[mm]

def next_month(mstr: str) -> str:
    yyyy = int(mstr[:4]); mm = int(mstr[-2:]) + 1
    if mm == 13: yyyy += 1; mm = 1
    return f"{yyyy}M{mm:02d}"

def months_between(start_m: str, end_m: str):
    out, cur = [], start_m
    while True:
        out.append(cur)
        if cur == end_m: break
        cur = next_month(cur)
    return out

# -------------------------------
# Build month window (inclusive)
# -------------------------------
months_mw = months_between(monthwide_start_m, monthwide_end_m)

# Common SQL fragment to normalize month (used in src CTE and filters)
NORMALIZE = "concat(substr(month,1,4),'M',lpad(regexp_extract(month,'M(\\\\d+)',1),2,'0'))"

# 1) month_wide per TEAM (uses base_cnos)
mw_cols_team_list = []
for m in months_mw:
    lab = month_to_label(m)  # JULY, AUG, SEP, ...
    mw_cols_team_list.append(
        f"SUM(CASE WHEN norm_month = '{m}' THEN cnos END)/1000/1000/150.1 AS {lab}_MM"
    )
    mw_cols_team_list.append(
        f"SUM(CASE WHEN norm_month = '{m}' THEN cnos END) / "
        f"NULLIF(SUM(CASE WHEN norm_month = '{m}' THEN cnos_ya END),0) * 100 AS {lab}_IYA"
    )
mw_cols_team_sql = ",\n    ".join(mw_cols_team_list)

# 2) 1H_to_date window (TEAM) — keep internal names as 1H_* (only output aliases change)
in_list = ",".join([f"'{m}'" for m in months_mw])
oneH_to_date_mm_sql  = (
    f"SUM(CASE WHEN norm_month IN ({in_list}) THEN cnos END)/1000/1000/150.1 AS 1H_to_date"
)
oneH_to_date_iya_sql = (
    f"SUM(CASE WHEN norm_month IN ({in_list}) THEN cnos END) / "
    f"NULLIF(SUM(CASE WHEN norm_month IN ({in_list}) THEN cnos_ya END),0) * 100 AS 1H_to_date_IYA"
)

# 3) month_wide for TOTAL (directly on src)
mw_cols_total_list = []
for m in months_mw:
    lab = month_to_label(m)
    mw_cols_total_list.append(
        f"SUM(CASE WHEN norm_month = '{m}' AND account='C-NOS' THEN forecast_act_est_d END)/1000/1000/150.1 AS {lab}_MM"
    )
    mw_cols_total_list.append(
        f"SUM(CASE WHEN norm_month = '{m}' AND account='C-NOS' THEN forecast_act_est_d END) / "
        f"NULLIF(SUM(CASE WHEN norm_month = '{m}' AND account='C-NOS' THEN ya_actual_d END),0) * 100 AS {lab}_IYA"
    )
mw_cols_total_sql = ",\n    ".join(mw_cols_total_list)

t_oneH_to_date_mm_sql  = (
    f"SUM(CASE WHEN norm_month IN ({in_list}) AND account='C-NOS' THEN forecast_act_est_d END)/1000/1000/150.1 AS 1H_to_date"
)
t_oneH_to_date_iya_sql = (
    f"SUM(CASE WHEN norm_month IN ({in_list}) AND account='C-NOS' THEN forecast_act_est_d END) / "
    f"NULLIF(SUM(CASE WHEN norm_month IN ({in_list}) AND account='C-NOS' THEN ya_actual_d END),0) * 100 AS 1H_to_date_IYA"
)

# 4) Aliases in team_metrics / total_metrics (JULY_MM -> JUL_MM, etc.)
mw_labels = [month_to_label(m) for m in months_mw]
team_month_mm_select_sql  = ", ".join([f"m.{lab}_MM AS {lab[:3]}_MM" for lab in mw_labels])
team_month_iya_select_sql = ", ".join([f"m.{lab}_IYA AS {lab[:3]}_IYA" for lab in mw_labels])

# 5) GROUP BY stability list (exact names from month_wide)
groupby_month_cols_sql = ", ".join([f"m.{lab}_MM, m.{lab}_IYA" for lab in mw_labels])

# 6) GIV to-date window (on src) — keep internal name as GIV_1H_* (only output alias changes)
giv_1h_to_date_sql = (
    f"SUM(CASE WHEN norm_month IN ({in_list}) THEN giv END) / "
    f"NULLIF(SUM(CASE WHEN norm_month IN ({in_list}) THEN giv_ya END),0) * 100 AS GIV_1H_to_date_IYA"
)

# 7) Final select month pairs (MM then IYA for each month) — prefix with c.
final_month_pairs_sql = ", ".join(
    sum([[f"ROUND(c.{lab[:3]}_MM, 0)  AS {lab[:3]}_MM",
          f"ROUND(c.{lab[:3]}_IYA, 0) AS {lab[:3]}_IYA"]
         for lab in mw_labels], [])
)

# Optional debug
print("Selected quarters:", fy_half_quarters)
print("quarter_in_sql:", quarter_in_sql)
print("Half label for output names:", half_label)

# -------------------------------
# Final SQL (all reads go via `src`, which defines `norm_month`)
# -------------------------------
query = f"""
WITH src AS (
  SELECT
    org2, org4, quarter, account,
    {NORMALIZE} AS norm_month,
    CAST(forecast_act_est AS DOUBLE) AS forecast_act_est_d,
    CAST(target           AS DOUBLE) AS target_d,
    CAST(ya_actual        AS DOUBLE) AS ya_actual_d
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE {NORMALIZE} BETWEEN '{start_m}' AND '{end_m}'
),

/* ------------ C‑NOS path (Div 1/Div 2 labels, no raw 'Division 2' detail) ------------ */
base_cnos_detail AS (
  SELECT
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END AS team,
    quarter,
    norm_month,
    SUM(CASE WHEN account = 'C-NOS' THEN forecast_act_est_d ELSE 0 END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN target_d           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN ya_actual_d        ELSE 0 END) AS cnos_ya
  FROM src
  WHERE quarter IN ({quarter_in_sql})
  GROUP BY
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END, quarter, norm_month
),
base_cnos_detail_clean AS (SELECT * FROM base_cnos_detail WHERE team IS NOT NULL),

base_cnos_div2 AS (
  SELECT
    'Div 2' AS team,
    quarter,
    norm_month,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est_d ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target_d           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual_d        ELSE 0 END) AS cnos_ya
  FROM src
  WHERE quarter IN ({quarter_in_sql}) AND org2 = 'Division 2'
  GROUP BY quarter, norm_month
),

base_cnos AS (
  SELECT * FROM base_cnos_detail_clean
  UNION ALL
  SELECT * FROM base_cnos_div2
),

quarter_rollup AS (
  SELECT
    team, quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY team, quarter
),
half_year AS (
  SELECT
    team,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
  GROUP BY team
),

/* Month-wide (parameterized, using norm_month) */
month_wide AS (
  SELECT
    team,
    {mw_cols_team_sql},
    {oneH_to_date_mm_sql},
    {oneH_to_date_iya_sql}
  FROM base_cnos
  GROUP BY team
),

team_metrics AS (
  SELECT
    m.team,
    {quarter_iya_sql},
    h.hy_target/1000/1000/150.1 AS `1H_MM`,
    h.hy_target/NULLIF(h.hy_ya,0)*100 AS `1H_IYA`,
    {team_month_mm_select_sql},
    {team_month_iya_select_sql},
    m.1H_to_date      AS `1H_To_Date_MM`,
    m.1H_to_date_IYA  AS `1H_To_Date_IYA`
  FROM month_wide m
  LEFT JOIN quarter_rollup q ON q.team = m.team
  LEFT JOIN half_year      h ON h.team = m.team
  GROUP BY m.team, h.hy_target, h.hy_cnos, h.hy_ya,
           {groupby_month_cols_sql},
           m.1H_to_date, m.1H_to_date_IYA
),

/* TOTAL path built on src */
t_quarter AS (
  SELECT
    'TOTAL' AS team,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est_d ELSE 0 END) AS q_cnos,
    SUM(CASE WHEN account='C-NOS' THEN target_d           ELSE 0 END) AS q_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual_d        ELSE 0 END) AS q_ya
  FROM src
  WHERE quarter IN ({quarter_in_sql})
  GROUP BY quarter
),
t_half_year AS (
  SELECT
    'TOTAL' AS team,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est_d ELSE 0 END) AS hy_cnos,
    SUM(CASE WHEN account='C-NOS' THEN target_d           ELSE 0 END) AS hy_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual_d        ELSE 0 END) AS hy_ya
  FROM src
),
t_month_wide AS (
  SELECT
    'TOTAL' AS team,
    {mw_cols_total_sql},
    {t_oneH_to_date_mm_sql},
    {t_oneH_to_date_iya_sql}
  FROM src
),

total_metrics AS (
  SELECT
    m.team,
    {quarter_iya_sql},
    h.hy_target/1000/1000/150.1 AS `1H_MM`,
    h.hy_target/NULLIF(h.hy_ya,0)*100 AS `1H_IYA`,
    {team_month_mm_select_sql},
    {team_month_iya_select_sql},
    m.1H_to_date      AS `1H_To_Date_MM`,
    m.1H_to_date_IYA  AS `1H_To_Date_IYA`
  FROM t_month_wide m
  LEFT JOIN t_quarter q ON q.team = m.team
  LEFT JOIN t_half_year h ON h.team = m.team
  GROUP BY m.team, h.hy_target, h.hy_cnos, h.hy_ya,
           {groupby_month_cols_sql},
           m.1H_to_date, m.1H_to_date_IYA
),

/* ------------ GIV path (to-date IYA) using src ------------ */
giv_base AS (
  SELECT
    org2, org4, norm_month,
    SUM(CASE WHEN account='GIV' THEN forecast_act_est_d END) AS giv,
    SUM(CASE WHEN account='GIV' THEN ya_actual_d        END) AS giv_ya
  FROM src
  GROUP BY org2, org4, norm_month
),
giv_detail AS (
  SELECT
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END AS team,
    norm_month,
    SUM(giv)    AS giv,
    SUM(giv_ya) AS giv_ya
  FROM giv_base
  GROUP BY
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END, norm_month
),
giv_detail_clean AS (SELECT * FROM giv_detail WHERE team IS NOT NULL),
giv_div2_total AS (
  SELECT 'Div 2' AS team, norm_month, SUM(giv) AS giv, SUM(giv_ya) AS giv_ya
  FROM giv_base
  WHERE org2 = 'Division 2'
  GROUP BY norm_month
),
giv_total AS (
  SELECT 'TOTAL' AS team, norm_month, SUM(giv) AS giv, SUM(giv_ya) AS giv_ya
  FROM giv_base
  GROUP BY norm_month
),
giv_all AS (
  SELECT * FROM giv_detail_clean
  UNION ALL SELECT * FROM giv_div2_total
  UNION ALL SELECT * FROM giv_total
),
giv_final AS (
  SELECT
    team,
    {giv_1h_to_date_sql}
  FROM giv_all
  GROUP BY team
),

/* ---- UNION team + total before final output ---- */
final_union_cnos AS (
  SELECT * FROM team_metrics
  UNION ALL
  SELECT * FROM total_metrics
)

/* ------------ Final output (includes TOTAL as first row) ------------ */
SELECT
  c.team AS `By Team`,
  {final_quarter_cols_sql},
  {final_half_summary_sql},
  {final_month_pairs_sql},
  {final_half_tail_sql}
FROM final_union_cnos c
LEFT JOIN giv_final g
  ON g.team = c.team
ORDER BY
  CASE c.team
    WHEN 'TOTAL'   THEN 0
    WHEN 'Div 1'   THEN 1
    WHEN 'Div 2'   THEN 2
    WHEN '   DP'   THEN 3
    WHEN '   MHC'  THEN 4
    WHEN '   HSM'  THEN 5
    WHEN '   WS'   THEN 6
    WHEN 'D-Com'   THEN 7
    WHEN 'Electro' THEN 8
    ELSE 99
  END, c.team ASC
"""

df = spark.sql(query)
display(df)

In [0]:
# %python
# -------------------------------
# Parameters (EDIT THESE)
# -------------------------------
start_m            = "2026M01"   # target overall filter start (inclusive)2026M01
end_m              = "2026M06"   # target overall filter end   (inclusive)
monthwide_start_m  = "2026M01"   # cnos month pivot start (drives JUL/AUG/... columns)
monthwide_end_m    = "2026M01"   # cnos month pivot end

# -------------------------------
# Dynamic FY Half -> Quarters + HALF label (NEW)
# -------------------------------
def month_num(mstr: str) -> int:
    return int(mstr[-2:])  # "2025M12" -> 12

_selected_month_num = month_num(end_m)

# FY quarters mapping:
# FY 1st half (Jul-Dec)  -> JAS, OND
# FY 2nd half (Jan-Jun)  -> JFM, AMJ
if 1 <= _selected_month_num <= 6:
    fy_half_quarters = ["JFM", "AMJ"]   # FY 2H
else:
    fy_half_quarters = ["JAS", "OND"]   # FY 1H

quarter_in_sql = ", ".join([f"'{q}'" for q in fy_half_quarters])

# Dynamic quarter IYA expressions for q_pivot
quarter_iya_sql = ",\n    ".join([
    f"MAX(CASE WHEN quarter = '{q}' THEN cnos_target / NULLIF(cnos_ya,0) * 100 END) AS {q}_IYA"
    for q in fy_half_quarters
])

# Dynamic quarter output columns (labeled select) - keep order in fy_half_quarters
final_quarter_cols_sql = ",\n    ".join([
    f"ROUND(f.{q}_IYA, 0) AS {q}_IYA"
    for q in fy_half_quarters
])

# Dynamic half label for OUTPUT NAMES ONLY (logic still uses internal 1H_* fields)
half_label = "2H" if (1 <= _selected_month_num <= 6) else "1H"

# HALF summary columns (must appear before month columns)
final_half_summary_sql = ",\n    ".join([
    f"ROUND(f.`1H_MM`, 0)  AS `{half_label}_MM`",
    f"ROUND(f.`1H_IYA`, 0) AS `{half_label}_IYA`"
])

# HALF tail columns (must appear after month columns)
final_half_tail_sql = ",\n    ".join([
    f"ROUND(f.`1H_To_Date_MM`, 0)  AS `{half_label}_To_Date_MM`",
    f"ROUND(f.`1H_to_date_IYA`, 0) AS `{half_label}_To_Date_IYA`",
    f"ROUND(f.`1H_to_date_IYA` - f.`1H_IYA`, 0) AS `vs_Target`",
    f"ROUND(g.GIV_1H_to_date_IYA, 0) AS `GIV_{half_label}_to_date_IYA`",
    f"ROUND(f.`1H_to_date_IYA` - g.GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`"
])

# -------------------------------
# Helpers
# -------------------------------
def month_to_label(mstr: str) -> str:
    mm = mstr[-2:]
    return {
        "01": "JAN", "02": "FEB", "03": "MAR", "04": "APR",
        "05": "MAY", "06": "JUN", "07": "JULY", "08": "AUG",
        "09": "SEP", "10": "OCT", "11": "NOV", "12": "DEC"
    }[mm]

def next_month(mstr: str) -> str:
    yyyy = int(mstr[:4]); mm = int(mstr[-2:]) + 1
    if mm == 13: yyyy += 1; mm = 1
    return f"{yyyy}M{mm:02d}"

def months_between(start_m: str, end_m: str):
    out, cur = [], start_m
    while True:
        out.append(cur)
        if cur == end_m: break
        cur = next_month(cur)
    return out

# Build month window (inclusive) for the pivot/to-date
months_mw = months_between(monthwide_start_m, monthwide_end_m)

# Common fragment to normalize the month key
NORMALIZE = "concat(substr(month,1,4),'M',lpad(regexp_extract(month,'M(\\\\d+)',1),2,'0'))"

# -------------------------------
# Build dynamic SQL fragments
# -------------------------------

# Monthly pivot (per-customer) dynamic columns
mw_cols_mpivot_list = []
for m in months_mw:
    lab = month_to_label(m)  # JULY, AUG, SEP, ...
    mw_cols_mpivot_list.append(
        f"MAX(CASE WHEN month = '{m}' THEN cnos END)/1000/1000/150.1 AS {lab}_MM"
    )
    mw_cols_mpivot_list.append(
        f"MAX(CASE WHEN month = '{m}' THEN cnos/cnos_ya*100 END)     AS {lab}_IYA"
    )
mw_cols_mpivot_sql = ",\n    ".join(mw_cols_mpivot_list)

# 1H-to-date (per-customer) using the pivot window (keep internal names as 1H_*)
in_list = ",".join([f"'{m}'" for m in months_mw])
oneH_to_date_mm_sql  = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END)/1000/1000/150.1 AS `1H_To_Date_MM`"
)
oneH_to_date_iya_sql = (
    f"SUM(CASE WHEN month IN ({in_list}) THEN cnos END)/"
    f"NULLIF(SUM(CASE WHEN month IN ({in_list}) THEN cnos_ya END),0)*100 AS `1H_to_date_IYA`"
)

# Aliases for final select (JULY_MM -> JUL_MM, etc.)
mw_labels = [month_to_label(m) for m in months_mw]
final_month_pairs_sql = ", ".join(
    sum([[f"ROUND(f.{lab}_MM, 0)  AS {lab[:3]}_MM", f"ROUND(f.{lab}_IYA, 0) AS {lab[:3]}_IYA"]
         for lab in mw_labels], [])
)

# For selecting month columns from m_pivot into final (keep stable)
m_mm_cols_sql  = ", ".join([f"m.{lab}_MM"  for lab in mw_labels])
m_iya_cols_sql = ", ".join([f"m.{lab}_IYA" for lab in mw_labels])

# -------------------------------
# Compose the full SQL (parameterized)
# -------------------------------
query = f"""
WITH base AS (
  SELECT
    customer, org2, channel, quarter, account,
    {NORMALIZE} AS norm_month,
    CAST(forecast_act_est AS DOUBLE) AS forecast_act_est_d,
    CAST(target           AS DOUBLE) AS target_d,
    CAST(ya_actual        AS DOUBLE) AS ya_actual_d
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE {NORMALIZE} BETWEEN '{start_m}' AND '{end_m}'
    AND quarter IN ({quarter_in_sql})
    AND channel != 'Baby Specialty'
),

-- Filtered customers
div1_cust AS (
  SELECT customer, org2, norm_month AS month, quarter, account,
         forecast_act_est_d AS forecast_act_est, target_d AS target, ya_actual_d AS ya_actual
  FROM base
  WHERE org2 = 'Division 1'
    AND customer IN (
      'FN-WELCIA YAKKYOKU','FN-CAINZ(C003)','FN-MATSUKIYO_COCOKARA (B008)','FN-SUGI_YAKKYOKU(I015)',
      'FN-AEON_GROUP(JUSCOJ4)(C005)','FN-COSMOS_YAKUHIN(W030)','FN-COSTCO(A062)','FN-TSURUHA_HOKK+TOHO(E030)',
      'FN-DONKIHOTE(A094)','FN-SUN_DRUG(D005)','FN-TGN','FN-KYORINDO(H011)','FN-LADY_YAKKYOKU(K067)',
      'FN-KUSURINO FUKUTARO (G043)','FN-ELEVEN(L002)','FN-UNY(B009)','FN-SUNDAY(E015)'
    )
),
dcom_cust AS (
  SELECT customer, org2, norm_month AS month, quarter, account,
         forecast_act_est_d AS forecast_act_est, target_d AS target, ya_actual_d AS ya_actual
  FROM base
  WHERE org2 = 'D-Com'
    AND customer IN ('FN-AMAZON JAPAN','FN-RAKUTEN')
),

-- Customer-level monthly aggregation (C-NOS)
cust_month_c AS (
  SELECT
    customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM (
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, month
),

-- Org-level monthly rollup (built ONLY from filtered customers)
org_month AS (
  SELECT
    'Division 1' AS customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM div1_cust
  GROUP BY month

  UNION ALL

  SELECT
    'D-Com' AS customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM dcom_cust
  GROUP BY month
),

-- Customer-level quarterly aggregation (C-NOS)
cust_quarter_c AS (
  SELECT
    customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM (
    SELECT customer, quarter, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, quarter, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, quarter
),

-- Org-level quarterly rollup (built ONLY from filtered customers)
org_quarter AS (
  SELECT
    'Division 1' AS customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM div1_cust
  GROUP BY quarter

  UNION ALL

  SELECT
    'D-Com' AS customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN forecast_act_est ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN target           ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN ya_actual        ELSE 0 END) AS cnos_ya
  FROM dcom_cust
  GROUP BY quarter
),

-- Unified monthly & quarterly datasets (customer + org)
unified_month AS (
  SELECT * FROM cust_month_c
  UNION ALL
  SELECT * FROM org_month
),
unified_quarter AS (
  SELECT * FROM cust_quarter_c
  UNION ALL
  SELECT * FROM org_quarter
),

-- Quarterly pivot & 1H metrics (C-NOS) — quarter columns DYNAMIC
q_pivot AS (
  SELECT
    customer,
    {quarter_iya_sql},
    SUM(cnos_target) / 1000 / 1000 / 150.1 AS `1H_MM`,
    SUM(cnos_target) / NULLIF(SUM(cnos_ya),0) * 100 AS `1H_IYA`
  FROM unified_quarter
  GROUP BY customer
),

-- Monthly pivot & 1H-to-date (C-NOS) — parameterized months
m_pivot AS (
  SELECT
    customer,
    {mw_cols_mpivot_sql},
    {oneH_to_date_mm_sql},
    {oneH_to_date_iya_sql}
  FROM unified_month
  GROUP BY customer
),

-- GIV monthly (customer + org)
giv_month_c AS (
  SELECT
    customer,
    month,
    SUM(CASE WHEN account='GIV' THEN forecast_act_est ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN target           ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN ya_actual        ELSE 0 END) AS GIV_ya
  FROM (
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, month
),
giv_month_org AS (
  SELECT
    'Division 1' AS customer,
    month,
    SUM(CASE WHEN account='GIV' THEN forecast_act_est ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN target           ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN ya_actual        ELSE 0 END) AS GIV_ya
  FROM div1_cust
  GROUP BY month

  UNION ALL

  SELECT
    'D-Com' AS customer,
    month,
    SUM(CASE WHEN account='GIV' THEN forecast_act_est ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN target           ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN ya_actual        ELSE 0 END) AS GIV_ya
  FROM dcom_cust
  GROUP BY month
),
giv_unified AS (
  SELECT * FROM giv_month_c
  UNION ALL
  SELECT * FROM giv_month_org
),
giv_pivot AS (
  SELECT
    customer,
    SUM(CASE WHEN month IN ({in_list}) THEN GIV END) /
      NULLIF(SUM(CASE WHEN month IN ({in_list}) THEN GIV_ya END),0) * 100 AS GIV_1H_to_date_IYA
  FROM giv_unified
  GROUP BY customer
),

final AS (
  SELECT
    COALESCE(q.customer, m.customer) AS customer,
    {", ".join([f"q.{q}_IYA" for q in fy_half_quarters])},
    q.`1H_MM`, q.`1H_IYA`,
    {m_mm_cols_sql},
    {m_iya_cols_sql},
    m.`1H_To_Date_MM`, m.`1H_to_date_IYA`,
    (q.`1H_IYA` - m.`1H_to_date_IYA`) AS `1H_To_Date_VS_1H_Target`
  FROM q_pivot q
  FULL OUTER JOIN m_pivot m
    ON q.customer = m.customer
),

-- Label & Order exactly like your reference
labeled AS (
  SELECT
    CASE
      WHEN f.customer = 'Division 1'             THEN 'Div 1 (Ex. Baby Specialty)'
      WHEN f.customer = 'D-Com'                  THEN 'D Com'
      WHEN f.customer = 'FN-WELCIA YAKKYOKU'     THEN '   WELCIA'
      WHEN f.customer = 'FN-CAINZ(C003)'         THEN '   CAINZ'
      WHEN f.customer = 'FN-MATSUKIYO_COCOKARA (B008)' THEN '   MCC'
      WHEN f.customer = 'FN-SUGI_YAKKYOKU(I015)' THEN '   SUGI'
      WHEN f.customer = 'FN-AEON_GROUP(JUSCOJ4)(C005)' THEN '   AEON'
      WHEN f.customer = 'FN-COSMOS_YAKUHIN(W030)' THEN '   COSMOS'
      WHEN f.customer = 'FN-COSTCO(A062)'        THEN '   COSTCO'
      WHEN f.customer = 'FN-TSURUHA_HOKK+TOHO(E030)' THEN '   TSURUHA'
      WHEN f.customer = 'FN-DONKIHOTE(A094)'     THEN '   DONKI'
      WHEN f.customer = 'FN-SUN_DRUG(D005)'      THEN '   SUNDRUG'
      WHEN f.customer = 'FN-TGN'                 THEN '   TGN'
      WHEN f.customer = 'FN-KYORINDO(H011)'      THEN '   KYORINDO'
      WHEN f.customer = 'FN-LADY_YAKKYOKU(K067)' THEN '   LADY'
      WHEN f.customer = 'FN-KUSURINO FUKUTARO (G043)' THEN '   FUKUTARO'
      WHEN f.customer = 'FN-ELEVEN(L002)'        THEN '   ELEVEN'
      WHEN f.customer = 'FN-UNY(B009)'           THEN '   UNY'
      WHEN f.customer = 'FN-SUNDAY(E015)'        THEN '   SUNDAY'
      WHEN f.customer = 'FN-AMAZON JAPAN'        THEN '   Amazon'
      WHEN f.customer = 'FN-RAKUTEN'             THEN '   Rakuten'
      ELSE f.customer
    END AS `By Customer`,

    -- Dynamic quarter columns (JAS/OND or JFM/AMJ)
    {final_quarter_cols_sql},

    -- Dynamic half summary (1H_* renamed to 1H/2H)
    {final_half_summary_sql},

    -- Dynamic month pairs (JUL_MM/JUL_IYA, AUG_MM/AUG_IYA, ...)
    {final_month_pairs_sql},

    -- Dynamic half tail (To_Date + vs_Target + GIV + Gap)
    {final_half_tail_sql}

  FROM final f
  LEFT JOIN giv_pivot g
    ON f.customer = g.customer
)

SELECT *
FROM labeled
ORDER BY
  CASE `By Customer`
    WHEN 'Div 1 (Ex. Baby Specialty)' THEN 0
    WHEN '   WELCIA'   THEN 1
    WHEN '   CAINZ'    THEN 2
    WHEN '   MCC'      THEN 3
    WHEN '   SUGI'     THEN 4
    WHEN '   AEON'     THEN 5
    WHEN '   COSMOS'   THEN 6
    WHEN '   COSTCO'   THEN 7
    WHEN '   TSURUHA'  THEN 8
    WHEN '   DONKI'    THEN 9
    WHEN '   SUNDRUG'  THEN 10
    WHEN '   TGN'      THEN 11
    WHEN '   KYORINDO' THEN 12
    WHEN '   LADY'     THEN 13
    WHEN '   FUKUTARO' THEN 14
    WHEN '   ELEVEN'   THEN 15
    WHEN '   UNY'      THEN 16
    WHEN '   SUNDAY'   THEN 17
    WHEN 'D Com'       THEN 18
    WHEN '   Amazon'   THEN 19
    WHEN '   Rakuten'  THEN 20
    ELSE 99
  END,
  `By Customer`;
"""

# Execute
df = spark.sql(query)
display(df)